# Music Analysis

In [1]:
import re
import sys
import time
import json
import math
import random
import spotipy
import requests
import itertools
import collections
import pandas as pd
from tqdm import tqdm
import spotipy.util as util
from bs4 import BeautifulSoup
from collections import defaultdict
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
def get_track_features(client_id = None, client_secret = None):
    if not client_id or not client_secret:
        print("Please add your client_id and client_secret")
        print("For more information, please visit ")
        print("https://developer.spotify.com/my-applications/#!/applications")
        return 
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, \
                                   client_secret = client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    # Get track info
    artist_name = []
    track_name = []
    track_id = []
    popularity = []
    years = []

    for year in tqdm(range(1960, 2020)):
        try:
            track_results = sp.search(q='year:{0}'.format(year), type='track', limit=50)
        except:
            break
        while track_results:
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                popularity.append(t['popularity'])
                years.append(str(year))
            if track_results['tracks']['next']:
                try:
                    track_results = sp.next(track_results['tracks'])
                except:
                    break
            else:
                track_results = None
    print("Get {0} songs from spotify".format(len(track_id)))
    # Get featrues
    feature_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', \
                      'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature']

    audio_features = {key:[] for key in feature_keys}
    audio_features['artist_name'] = artist_name
    audio_features['track_name'] = track_name
    audio_features['track_id'] = track_id
    audio_features['popularity'] = popularity
    audio_features['year'] = years # list(itertools.chain(*[[i] * 10000 for i in range(1960, 2020)])) # years

    for i in tqdm(range(0, len(track_id), 50)):
        features = sp.audio_features(track_id[i: i + 50])  
        for feature in features:
            for key in feature_keys:
                if feature and key in feature:
                    audio_features[key].append(feature[key])
                else:
                    audio_features[key].append(None)
    
    # Save to csv file
    df_tracks = pd.DataFrame(audio_features)
    df_tracks.head()
    df_tracks.to_csv(r'audio_features.csv')

In [3]:
# We've already fetch all the data and corresponding features,
# and saved it in "./audio_features.csv"
# no need to run this function again
client_id = None
client_secret = None
get_track_features(client_id, client_secret)

Please add your client_id and client_secret
For more information, please visit 
https://developer.spotify.com/my-applications/#!/applications


In [4]:
df_tracks = pd.read_csv('audio_features.csv', index_col=0)
df_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,track_id,popularity,year
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.707,0.00381,0.302,0.315,174.431,182400.0,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.764,0.00000,0.153,0.644,154.759,175987.0,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.733,0.00000,0.281,0.836,76.816,131733.0,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.878,0.00000,0.129,0.407,96.217,165560.0,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.617,0.00000,0.155,0.609,70.843,165280.0,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960


In [5]:
df_by_year = df_tracks.dropna().groupby('year').mean().reset_index()
df_by_year.head()

,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
0,1960,0.464192,0.313453,5.005300,-14.693014,0.715200,0.100909,0.809230,0.253353,0.222327,0.494803,110.969244,215520.814900,3.750900,5.448100
1,1961,0.474276,0.320147,4.948585,-14.829432,0.720816,0.107911,0.799795,0.258537,0.218472,0.493421,110.856446,222618.542363,3.752826,5.115635
2,1962,0.463054,0.330539,5.049005,-14.832738,0.719772,0.100380,0.782238,0.254475,0.225328,0.484301,111.210020,214440.192719,3.749175,6.244124
3,1963,0.478339,0.357190,5.090700,-13.994034,0.742600,0.078978,0.758253,0.238693,0.220458,0.526543,112.315946,202682.442800,3.769200,7.717000
4,1964,0.478219,0.371290,5.084208,-13.754991,0.747575,0.080756,0.746077,0.217783,0.228598,0.528866,113.116872,198727.756076,3.767377,8.243324


In [6]:
import plotly.graph_objects as go
from ipywidgets import widgets
from ipywidgets import interactive
import plotly.express as px

In [7]:
virtualize_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', \
                      'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature', 'popularity']

feature_box = widgets.Dropdown(options=virtualize_features,
                          value=virtualize_features[0],
                          description='Feature: ')

trace1 = go.Scatter(x=df_by_year['year'], y=df_by_year['danceability'])

g = go.FigureWidget(data=[trace1],
                   layout=go.Layout(
                        title=dict(
                            text='Music Features Over Years'
                        ),
                        xaxis=dict(
                            title="Year"
                        ),
                        yaxis=dict(
                            title="danceability"
                        ),
                    ))

def response(change):
    new_feature = change['new']
    g.layout.yaxis.title = new_feature
    g.data[0].y = df_by_year[new_feature]

feature_box.observe(response, names="value")

widgets.VBox([feature_box, g])

In [8]:
all_musicians = list(df_tracks['artist_name'].unique())
print(len(all_musicians))

53981


Fetch the birthplace of musicians using Wikidata SPARQL query

In [ ]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/
# https://stackoverflow.com/questions/45171646/how-to-get-birth-location-using-wikipedia-api
def search_birthlocation(musician):
    from SPARQLWrapper import SPARQLWrapper, JSON
    endpoint_url = "https://query.wikidata.org/sparql"
    query = """SELECT DISTINCT ?item ?itemLabel ?birthLocation ?birthLocationLabel WHERE {{
              ?item (wdt:P31|wdt:P101|wdt:P106)/wdt:P279* wd:Q639669 ;
            rdfs:label "{0}"@en ;wdt:P19 ?birthLocation
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}""".format(musician)
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        query_result = sparql.query().convert()
    except:
        musician_birth['birthplace'].append("NaN")
        return
    if not query_result["results"]["bindings"]:
        musician_birth['birthplace'].append("NaN")
    else:
        result = query_result["results"]["bindings"][0]
        if 'birthLocationLabel' in result:
            musician_birth['birthplace'].append(result['birthLocationLabel']['value'])
        else:
            musician_birth['birthplace'].append("None")

def get_musician_birth_by_wikisql(run = False):
    if not run:
        return 
    musician_birth = {key:[] for key in ['name', 'birthplace']}

    for i in tqdm(all_musicians):
        time.sleep(0.1)
        musician_birth['name'].append(i)
        search_birthlocation(i)

    print(musician_birth)
    df_musician = pd.DataFrame(musician_birth)
    df_musician.head()
    df_musician.to_csv(r'musician_birth.csv', index=[0])
    

# We've already ran this funtion and saved result in "./musician_birth.csv"
# no need to run this function again because it will take several hours.

# get_musician_birth_by_wikisql()

In [10]:
df_musician = pd.read_csv('musician_birth.csv', index_col=0)
df_musician.head()

,name,birthplace
0,Etta James,Los Angeles
1,Ella Fitzgerald,Newport News
2,Sam Cooke,NaN
3,Neil Sedaka,Brooklyn
4,Dean Martin,Steubenville


Some musicians don't have birthplace record in Wikidata database, so we move to fetch the birthplace of remained musicians by scraping Wikipedia page directly. 

In [11]:
def get_musician_birth_by_wiki(run = False):
    if not run:
        return 
    null = 0
    find = 0
    for i, name in enumerate(df_musician.name):
        if i != 0 and i % 1000 == 0:
            print(i)
            print(find)
            print(null)
        if i < 34369:
            continue
        if type(df_musician.loc[i, "birthplace"]) != float or not math.isnan(df_musician.loc[i, "birthplace"]):
            continue
        null += 1
        url = "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xmlfm&titles={0}&rvsection=0".format(name)
        r = requests.get(url)
        soup = BeautifulSoup(r._content, 'html.parser')
        all_matches = re.findall(r"birth_place [^\n]+\n\|", soup.prettify())
        if len(all_matches) == 0:
            all_matches = re.findall(r"origin [^\n]+\n\|", soup.prettify())
        if len(all_matches):
            df_musician.loc[i, "birthplace"] = all_matches[0].strip('\n|').replace('[', '').replace(']', '').split('=')[1].strip()
            find += 1
        time.sleep(random.uniform(0.1, 0.4))
        df_musician.to_csv(r'musician_birth1.csv', index=[0])

# We've already ran this funtion and saved result in "./musician_birth1.csv"
# no need to run this function again because it will take several hours.

# get_musician_birth_by_wiki()

In [15]:
df_musician = pd.read_csv('musician_birth1.csv', index_col=0)
df_musician.head()
print(len(df_musician[df_musician['birthplace'].isnull()]))
new_df = df_musician[df_musician['birthplace'].isnull()]
l = set(list(new_df.name))
print(len(df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)]))
# df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)].dropna().groupby('year').count()

33403
400205


Now, there are still 33403 musicians which don't have birthplace information. So we try to add "(singer)" or "(band)" at the end of the musician name when scraping the Wikipedia because of the duplication of name.

Also, to save time, we only query unknown birthplace musicians who have more than 10 songs, the number of them is 3633.

In [17]:
df = df_tracks[df_tracks['artist_name'].apply(lambda x : x in l)]
df = df.groupby('artist_name').size().sort_values()
df = df > 10
df = df[df]
names = ([name for name in df.index])
print(len(names))

3633


In [ ]:
res = {}
def get_musician_birth_by_wiki(run = False):
    if not run:
        return 
    null = 0
    find = 0
    for name in tqdm(names):
        null += 1
        base_url = "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xmlfm&titles={0}&rvsection=0".format(name)
        singer_url = base_url + " (singer)"
        band_url = base_url + " (band)"
        for url in [singer_url, band_url]:
            r = requests.get(url)
            soup = BeautifulSoup(r._content, 'html.parser')
            all_matches = re.findall(r"birth_place [^\n]+\n\|", soup.prettify())
            if len(all_matches) == 0:
                all_matches = re.findall(r"origin [^\n]+\n\|", soup.prettify())
            if len(all_matches):
                res[name] = all_matches[0].strip('\n|').replace('[', '').replace(']', '').split('=')[1].strip()
                find += 1
                break
        time.sleep(random.uniform(0.1, 0.4))
    print(len(res))
    df_musician = pd.read_csv('musician_birth1.csv', index_col=0)
    df_musician.head()
    for i, name in enumerate(df_musician.name):
        if name in res:
            df_musician.loc[i, 'birthplace'] = res[name]
    df_musician.to_csv(r'musician_birth_final.csv', index=[0])

# We've already ran this funtion and saved result in "./musician_birth_final.csv"
# no need to run this function again because it will take several hours.
get_musician_birth_by_wiki(run = False)                                                                                                                         

After that, we have successfully fetch birthplace of 709 more musicians and update these information into dataframe. And the number of tracks composed by known birthplace artists has increased to 430K.

In [20]:
df_musician = pd.read_csv('musician_birth_final.csv', index_col=0)
df_musician.head()
df_musician = df_musician.drop(columns=['birth_place'])
print(len(df_musician[df_musician['birthplace'].isnull()]))
new_df = df_musician[df_musician['birthplace'].isnull()]
l = set(list(new_df.name))
print(len(df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)]))

32719
430865


Right now, we have got the birthplace information of musicians birthplace. The next step is to fetch the latitude and longtitude of these birthplaces because we want to draw these places on the map.

In [46]:
def fetch_lat_lon(location):
    url = "http://open.mapquestapi.com/nominatim/v1/search.php"

    data = {
        'key': '3q2AMpu1zq1lJ4dVGpSZVtWB8TugZc9J',
        'q': location,
        'format': 'json',
        'limit': 1
    }

    response = requests.get(url, params=data)

    response_json = json.loads(response.text)
    
    if len(response_json) < 1:
        return None, None
    else:
        return response_json[0]['lat'], response_json[0]['lon']

# print(fetch_lat_lon("Los Angeles"))

# lat_res = []
# lon_res = []
# for location in tqdm(df_musician['birthplace']):
#     if type(location) is str:
#         time.sleep(0.1)
#         try:
#             lat, lon = fetch_lat_lon(location)
#         except:
#             print(location)
#             break
#         lat_res.append(lat)
#         lon_res.append(lon)
#     else:
#         lat_res.append(None)
#         lon_res.append(None)


In [33]:
url = "http://open.mapquestapi.com/nominatim/v1/search.php?key=UGS76tyYoUNTDngBGwdT45gJ1jGRWcbp&format=json&q=Linkou District&limit=1"

response = requests.get(url)

response_json = json.loads(response.text)

print(response_json)

[{'place_id': '187745809', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://www.openstreetmap.org/copyright', 'osm_type': 'relation', 'osm_id': '2878928', 'boundingbox': ['25.0624957', '25.1369185', '121.2826336', '121.4150345'], 'lat': '25.0775308', 'lon': '121.3916021', 'display_name': '林口區, 新北市, 臺灣', 'class': 'boundary', 'type': 'administrative', 'importance': 0.096103981157114, 'icon': 'http://ip-10-116-133-155.mq-us-west-2.ec2.aolcloud.net/nominatim/images/mapicons/poi_boundary_administrative.p.20.png'}]


In [43]:
print(len([i for i in lat_res if i]), len(lat_res))
# for i, location in enumerate(df_musician['birthplace']):
#     if type(location) is str and not lat_res[i]:
#         print(i, location)
#         break
print(lat_res[-5:])

10050 28183
[None, None, '53.902334', '40.643715', None]


In [44]:
for location in tqdm(df_musician['birthplace'][28183:]):
    if type(location) is str:
        time.sleep(0.1)
        try:
            lat, lon = fetch_lat_lon(location)
        except:
            print(location)
            break
        lat_res.append(lat)
        lon_res.append(lon)
    else:
        lat_res.append(None)
        lon_res.append(None)



  0%|          | 0/25798 [00:00<?, ?it/s]

  0%|          | 1/25798 [00:00<3:25:38,  2.09it/s]

  0%|          | 3/25798 [00:00<2:40:20,  2.68it/s]

  0%|          | 4/25798 [00:01<2:30:18,  2.86it/s]

  0%|          | 5/25798 [00:01<2:17:56,  3.12it/s]

  0%|          | 6/25798 [00:01<2:09:15,  3.33it/s]

  0%|          | 7/25798 [00:01<2:01:44,  3.53it/s]

  0%|          | 9/25798 [00:02<1:39:45,  4.31it/s]

  0%|          | 12/25798 [00:02<1:18:15,  5.49it/s]

  0%|          | 14/25798 [00:02<1:09:54,  6.15it/s]

  0%|          | 15/25798 [00:02<1:18:15,  5.49it/s]

  0%|          | 16/25798 [00:02<1:30:15,  4.76it/s]

  0%|          | 24/25798 [00:04<1:34:28,  4.55it/s]

  0%|          | 25/25798 [00:05<1:37:07,  4.42it/s]

  0%|          | 27/25798 [00:05<1:20:22,  5.34it/s]

  0%|          | 29/25798 [00:05<1:10:42,  6.07it/s]

  0%|          | 30/25798 [00:05<1:18:42,  5.46it/s]

  0%|          | 41/25798 [00:06<58:22,  7.35it/s]  

  0%|          | 44/25798 [00:06<53:06,  8.0

  2%|▏         | 626/25798 [01:25<1:16:38,  5.47it/s]

  2%|▏         | 627/25798 [01:26<1:23:41,  5.01it/s]

  2%|▏         | 628/25798 [01:26<1:28:53,  4.72it/s]

  2%|▏         | 630/25798 [01:26<1:17:40,  5.40it/s]

  2%|▏         | 631/25798 [01:26<1:24:57,  4.94it/s]

  2%|▏         | 632/25798 [01:27<1:36:34,  4.34it/s]

  2%|▏         | 633/25798 [01:27<1:31:09,  4.60it/s]

  2%|▏         | 635/25798 [01:27<1:18:50,  5.32it/s]

  2%|▏         | 637/25798 [01:27<1:09:21,  6.05it/s]

  2%|▏         | 638/25798 [01:28<1:17:20,  5.42it/s]

  2%|▏         | 639/25798 [01:28<1:23:53,  5.00it/s]

  2%|▏         | 642/25798 [01:28<1:11:11,  5.89it/s]

  2%|▏         | 643/25798 [01:28<1:18:07,  5.37it/s]

  2%|▏         | 644/25798 [01:29<1:23:05,  5.05it/s]

  3%|▎         | 645/25798 [01:29<1:35:05,  4.41it/s]

  3%|▎         | 646/25798 [01:29<1:37:14,  4.31it/s]

  3%|▎         | 649/25798 [01:29<1:16:13,  5.50it/s]

  3%|▎         | 650/25798 [01:29<1:24:38,  4.95it/s]

  3%|▎    

  5%|▍         | 1251/25798 [02:51<47:58,  8.53it/s]

  5%|▍         | 1253/25798 [02:51<49:00,  8.35it/s]

  5%|▍         | 1255/25798 [02:52<48:32,  8.43it/s]

  5%|▍         | 1257/25798 [02:52<1:01:41,  6.63it/s]

  5%|▍         | 1258/25798 [02:52<1:16:34,  5.34it/s]

  5%|▍         | 1259/25798 [02:53<1:23:31,  4.90it/s]

  5%|▍         | 1260/25798 [02:53<1:26:17,  4.74it/s]

  5%|▍         | 1263/25798 [02:53<1:09:24,  5.89it/s]

  5%|▍         | 1265/25798 [02:53<1:02:45,  6.52it/s]

  5%|▍         | 1266/25798 [02:53<1:12:28,  5.64it/s]

  5%|▍         | 1269/25798 [02:54<1:00:30,  6.76it/s]

  5%|▍         | 1271/25798 [02:54<57:31,  7.11it/s]  

  5%|▍         | 1273/25798 [02:54<54:57,  7.44it/s]

  5%|▍         | 1275/25798 [02:54<50:24,  8.11it/s]

  5%|▍         | 1277/25798 [02:55<49:49,  8.20it/s]

  5%|▍         | 1278/25798 [02:55<58:19,  7.01it/s]

  5%|▍         | 1279/25798 [02:55<1:35:21,  4.29it/s]

  5%|▍         | 1280/25798 [02:55<1:34:32,  4.32it/s]

  5%|▍

KeyboardInterrupt: 

In [47]:
print(len([i for i in lat_res if i]), len(lat_res))
for location in tqdm(df_musician['birthplace'][29587:]):
    if type(location) is str:
        time.sleep(0.1)
        try:
            lat, lon = fetch_lat_lon(location)
        except:
            print(location)
            break
        lat_res.append(lat)
        lon_res.append(lon)
    else:
        lat_res.append(None)
        lon_res.append(None)




  0%|          | 0/24394 [00:00<?, ?it/s]

10558 29587





  0%|          | 1/24394 [00:00<1:35:36,  4.25it/s]


  0%|          | 2/24394 [00:00<1:35:46,  4.24it/s]


  0%|          | 5/24394 [00:00<1:16:26,  5.32it/s]


  0%|          | 6/24394 [00:00<1:26:35,  4.69it/s]


  0%|          | 7/24394 [00:01<1:27:40,  4.64it/s]


  0%|          | 8/24394 [00:01<1:32:52,  4.38it/s]


  0%|          | 9/24394 [00:01<1:34:38,  4.29it/s]


  0%|          | 10/24394 [00:01<1:29:16,  4.55it/s]


  0%|          | 11/24394 [00:02<1:33:12,  4.36it/s]


  0%|          | 12/24394 [00:02<1:46:56,  3.80it/s]


  0%|          | 13/24394 [00:02<2:01:51,  3.33it/s]


  0%|          | 14/24394 [00:03<2:55:41,  2.31it/s]


  0%|          | 15/24394 [00:03<2:47:18,  2.43it/s]


  0%|          | 16/24394 [00:04<2:48:55,  2.41it/s]


  0%|          | 17/24394 [00:05<3:46:03,  1.80it/s]


  0%|          | 18/24394 [00:05<3:22:29,  2.01it/s]


  0%|          | 21/24394 [00:05<2:32:59,  2.66it/s]


  0%|          | 23/24394 [00:06<2:06:59,  3.20it/s]


  0%|         

  2%|▏         | 547/24394 [01:16<1:06:13,  6.00it/s]


  2%|▏         | 548/24394 [01:16<1:17:45,  5.11it/s]


  2%|▏         | 549/24394 [01:17<1:20:45,  4.92it/s]


  2%|▏         | 550/24394 [01:17<1:19:42,  4.99it/s]


  2%|▏         | 553/24394 [01:17<1:06:52,  5.94it/s]


  2%|▏         | 554/24394 [01:17<1:14:39,  5.32it/s]


  2%|▏         | 558/24394 [01:18<59:33,  6.67it/s]  


  2%|▏         | 560/24394 [01:18<55:28,  7.16it/s]


  2%|▏         | 563/24394 [01:18<47:46,  8.31it/s]


  2%|▏         | 567/24394 [01:18<40:35,  9.79it/s]


  2%|▏         | 569/24394 [01:19<42:15,  9.40it/s]


  2%|▏         | 571/24394 [01:19<43:24,  9.15it/s]


  2%|▏         | 579/24394 [01:19<33:49, 11.74it/s]


  2%|▏         | 583/24394 [01:19<31:08, 12.74it/s]


  2%|▏         | 585/24394 [01:20<52:45,  7.52it/s]


  2%|▏         | 587/24394 [01:20<52:15,  7.59it/s]


  2%|▏         | 590/24394 [01:20<45:50,  8.66it/s]


  2%|▏         | 592/24394 [01:21<48:38,  8.16it/s]


  2%|▏        

  5%|▍         | 1147/24394 [02:28<1:13:17,  5.29it/s]


  5%|▍         | 1148/24394 [02:29<1:20:05,  4.84it/s]


  5%|▍         | 1152/24394 [02:29<1:02:47,  6.17it/s]


  5%|▍         | 1153/24394 [02:29<1:11:53,  5.39it/s]


  5%|▍         | 1155/24394 [02:29<1:05:04,  5.95it/s]


  5%|▍         | 1156/24394 [02:30<1:14:45,  5.18it/s]


  5%|▍         | 1158/24394 [02:30<1:06:33,  5.82it/s]


  5%|▍         | 1160/24394 [02:30<1:00:28,  6.40it/s]


  5%|▍         | 1161/24394 [02:30<1:13:22,  5.28it/s]


  5%|▍         | 1162/24394 [02:31<1:18:45,  4.92it/s]


  5%|▍         | 1163/24394 [02:31<1:36:30,  4.01it/s]


  5%|▍         | 1164/24394 [02:31<1:36:55,  3.99it/s]


  5%|▍         | 1168/24394 [02:32<1:13:28,  5.27it/s]


  5%|▍         | 1170/24394 [02:32<1:05:50,  5.88it/s]


  5%|▍         | 1171/24394 [02:32<1:14:32,  5.19it/s]


  5%|▍         | 1173/24394 [02:32<1:04:29,  6.00it/s]


  5%|▍         | 1174/24394 [02:32<1:12:03,  5.37it/s]


  5%|▍         | 1175/24394 [02

...

 72%|███████▏  | 17587/24394 [36:27<14:51,  7.64it/s]


 72%|███████▏  | 17588/24394 [36:27<18:27,  6.15it/s]


 72%|███████▏  | 17591/24394 [36:28<15:31,  7.30it/s]


 72%|███████▏  | 17594/24394 [36:28<13:26,  8.44it/s]


 72%|███████▏  | 17596/24394 [36:28<16:10,  7.00it/s]


 72%|███████▏  | 17598/24394 [36:28<14:31,  7.80it/s]


 72%|███████▏  | 17599/24394 [36:29<18:16,  6.20it/s]


 72%|███████▏  | 17602/24394 [36:29<15:29,  7.31it/s]


 72%|███████▏  | 17604/24394 [36:29<14:56,  7.58it/s]


 72%|███████▏  | 17605/24394 [36:29<18:28,  6.12it/s]


 72%|███████▏  | 17607/24394 [36:30<17:39,  6.41it/s]


 72%|███████▏  | 17609/24394 [36:30<16:19,  6.93it/s]


 72%|███████▏  | 17610/24394 [36:30<19:08,  5.91it/s]


 72%|███████▏  | 17612/24394 [36:30<17:13,  6.56it/s]


 72%|███████▏  | 17617/24394 [36:31<13:36,  8.30it/s]


 72%|███████▏  | 17619/24394 [36:31<13:37,  8.28it/s]


 72%|███████▏  | 17621/24394 [36:31<18:01,  6.26it/s]


 72%|███████▏  | 17626/24394 [36:32<13:51,  8.13

 76%|███████▌  | 18563/24394 [37:49<10:32,  9.22it/s]


 76%|███████▌  | 18565/24394 [37:50<18:31,  5.24it/s]


 76%|███████▌  | 18572/24394 [37:50<14:27,  6.71it/s]


 76%|███████▌  | 18575/24394 [37:51<13:28,  7.20it/s]


 76%|███████▌  | 18577/24394 [37:52<26:46,  3.62it/s]


 76%|███████▌  | 18578/24394 [37:52<25:28,  3.81it/s]


 76%|███████▌  | 18588/24394 [37:53<18:36,  5.20it/s]


 76%|███████▌  | 18596/24394 [37:53<13:50,  6.98it/s]


 76%|███████▌  | 18599/24394 [37:53<11:47,  8.19it/s]


 76%|███████▋  | 18602/24394 [37:53<10:24,  9.27it/s]


 76%|███████▋  | 18604/24394 [37:54<14:19,  6.74it/s]


 76%|███████▋  | 18612/24394 [37:54<10:49,  8.90it/s]


 76%|███████▋  | 18620/24394 [37:54<08:24, 11.44it/s]


 76%|███████▋  | 18623/24394 [37:54<07:48, 12.33it/s]


 76%|███████▋  | 18626/24394 [37:55<07:45, 12.40it/s]


 76%|███████▋  | 18631/24394 [37:55<06:50, 14.04it/s]


 76%|███████▋  | 18633/24394 [37:55<07:26, 12.90it/s]


 76%|███████▋  | 18645/24394 [37:55<05:46, 16.58

 81%|████████  | 19646/24394 [39:05<06:36, 11.98it/s]


 81%|████████  | 19653/24394 [39:06<05:22, 14.69it/s]


 81%|████████  | 19662/24394 [39:06<04:22, 18.04it/s]


 81%|████████  | 19665/24394 [39:06<05:00, 15.74it/s]


 81%|████████  | 19668/24394 [39:07<06:41, 11.76it/s]


 81%|████████  | 19670/24394 [39:07<09:59,  7.88it/s]


 81%|████████  | 19674/24394 [39:07<08:05,  9.73it/s]


 81%|████████  | 19676/24394 [39:08<11:58,  6.57it/s]


 81%|████████  | 19687/24394 [39:08<08:55,  8.79it/s]


 81%|████████  | 19696/24394 [39:08<06:59, 11.19it/s]


 81%|████████  | 19699/24394 [39:08<06:37, 11.81it/s]


 81%|████████  | 19706/24394 [39:09<05:25, 14.42it/s]


 81%|████████  | 19727/24394 [39:09<04:01, 19.31it/s]


 81%|████████  | 19732/24394 [39:09<03:55, 19.82it/s]


 81%|████████  | 19736/24394 [39:09<04:09, 18.68it/s]


 81%|████████  | 19741/24394 [39:10<04:00, 19.31it/s]


 81%|████████  | 19747/24394 [39:10<03:44, 20.66it/s]


 81%|████████  | 19750/24394 [39:10<06:20, 12.19

 86%|████████▋ | 21055/24394 [40:24<05:01, 11.08it/s]


 86%|████████▋ | 21066/24394 [40:24<03:57, 14.02it/s]


 86%|████████▋ | 21081/24394 [40:25<03:32, 15.58it/s]


 86%|████████▋ | 21084/24394 [40:25<04:29, 12.28it/s]


 86%|████████▋ | 21094/24394 [40:25<03:44, 14.67it/s]


 86%|████████▋ | 21099/24394 [40:26<03:41, 14.85it/s]


 87%|████████▋ | 21113/24394 [40:26<02:58, 18.35it/s]


 87%|████████▋ | 21119/24394 [40:26<02:45, 19.76it/s]


 87%|████████▋ | 21125/24394 [40:26<02:26, 22.26it/s]


 87%|████████▋ | 21155/24394 [40:27<01:49, 29.47it/s]


 87%|████████▋ | 21161/24394 [40:27<02:01, 26.58it/s]


 87%|████████▋ | 21169/24394 [40:27<02:06, 25.42it/s]


 87%|████████▋ | 21173/24394 [40:28<03:20, 16.05it/s]


 87%|████████▋ | 21178/24394 [40:28<03:07, 17.19it/s]


 87%|████████▋ | 21189/24394 [40:28<02:27, 21.78it/s]


 87%|████████▋ | 21193/24394 [40:28<02:37, 20.30it/s]


 87%|████████▋ | 21203/24394 [40:29<02:14, 23.79it/s]


 87%|████████▋ | 21207/24394 [40:29<02:32, 20.89

 94%|█████████▍| 22877/24394 [41:40<00:43, 34.56it/s]


 94%|█████████▍| 22883/24394 [41:41<00:48, 30.93it/s]


 94%|█████████▍| 22887/24394 [41:41<01:00, 25.07it/s]


 94%|█████████▍| 22891/24394 [41:41<01:34, 15.93it/s]


 94%|█████████▍| 22908/24394 [41:41<01:12, 20.59it/s]


 94%|█████████▍| 22912/24394 [41:42<01:20, 18.52it/s]


 94%|█████████▍| 22915/24394 [41:42<01:36, 15.29it/s]


 94%|█████████▍| 22920/24394 [41:42<01:24, 17.38it/s]


 94%|█████████▍| 22924/24394 [41:42<01:25, 17.10it/s]


 94%|█████████▍| 22930/24394 [41:43<01:17, 18.96it/s]


 94%|█████████▍| 22951/24394 [41:43<00:58, 24.66it/s]


 94%|█████████▍| 22956/24394 [41:43<01:01, 23.56it/s]


 94%|█████████▍| 22968/24394 [41:43<00:51, 27.88it/s]


 94%|█████████▍| 22972/24394 [41:44<01:00, 23.32it/s]


 94%|█████████▍| 22985/24394 [41:44<00:49, 28.32it/s]


 94%|█████████▍| 22993/24394 [41:44<00:46, 29.93it/s]


 94%|█████████▍| 23041/24394 [41:44<00:33, 40.29it/s]


 94%|█████████▍| 23050/24394 [41:45<00:45, 29.27

In [52]:
print(len([i for i in lat_res if i]))


17828


In [54]:
with open('lat_res.json', 'w') as outfile:
    json.dump(lat_res, outfile)
with open('lon_res.json', 'w') as outfile:
    json.dump(lon_res, outfile)

In [112]:
def fetch_lat_lon2(location):
    url = "https://us1.locationiq.com/v1/search.php"
    
    data = {
        'key': '291ba8bb10a30a',
        'q': location,
        'format': 'json',
    }

    response = requests.get(url, params=data)

    response_json = json.loads(response.text)
    
    if len(response_json) < 1:
        return None, None
    else:
        return response_json[0]['lat'], response_json[0]['lon']
        
        

In [65]:
df_musician['birthplace'][238] = 'Moscow, Russia'
print(df_musician['birthplace'][238])

Moscow, Russia


In [62]:

df_musician['birthplace'] = df_musician['birthplace'].apply(lambda x : x.replace('<span class', '') if type(x) is str else x)
df_musician['birthplace'] = df_musician['birthplace'].apply(lambda x : x.replace('{{nowrap|', '') if type(x) is str else x)


In [67]:
failed_location_dict = {}

for i, location in tqdm(enumerate(df_musician['birthplace'])):
    if type(location) is str and not lat_res[i]:
        time.sleep(1)
        try:
            lat, lon = fetch_lat_lon2(location)
        except:
            failed_location_dict[i] = location
            continue
        lat_res[i] = lat
        lon_res[i] = lon








0it [00:00, ?it/s]






671it [00:01, 568.10it/s]






689it [00:02, 40.94it/s] 






706it [00:04, 23.39it/s]






715it [00:05, 13.70it/s]






722it [00:06,  9.60it/s]






734it [00:07,  9.45it/s]






738it [00:09,  5.53it/s]






768it [00:10,  7.23it/s]






773it [00:12,  5.25it/s]






793it [00:13,  6.57it/s]






795it [00:15,  2.47it/s]






806it [00:18,  2.73it/s]






813it [00:19,  3.11it/s]






815it [00:21,  2.27it/s]






827it [00:22,  2.92it/s]






854it [00:24,  3.90it/s]






865it [00:25,  4.61it/s]






882it [00:26,  5.66it/s]






890it [00:28,  5.66it/s]






897it [00:29,  5.74it/s]






921it [00:31,  6.96it/s]






922it [00:32,  2.18it/s]






968it [00:33,  3.04it/s]






1011it [00:34,  4.19it/s]






1029it [00:36,  5.32it/s]






1031it [00:38,  2.48it/s]






1046it [00:39,  3.23it/s]






1078it [00:41,  4.24it/s]






1089it [00:42,  5.05it/s]






1093it [00:43,  4.25it/s]






1102it [00:45,  4.78it/s]





3508it [03:52,  2.48it/s]






3521it [03:54,  3.21it/s]






3522it [03:55,  1.59it/s]






3556it [03:56,  2.21it/s]






3561it [03:58,  2.33it/s]






3571it [04:00,  2.97it/s]






3581it [04:01,  3.48it/s]






3612it [04:03,  4.68it/s]






3617it [04:04,  4.54it/s]






3650it [04:05,  6.04it/s]






3681it [04:06,  7.77it/s]






3706it [04:08,  9.41it/s]






3726it [04:09, 10.80it/s]






3810it [04:10, 14.31it/s]






3813it [04:12,  5.88it/s]






3853it [04:13,  7.81it/s]






3858it [04:14,  5.88it/s]






3884it [04:16,  7.13it/s]






3887it [04:17,  4.17it/s]






3895it [04:19,  4.70it/s]






3936it [04:20,  6.24it/s]






3957it [04:21,  7.57it/s]






3996it [04:23,  9.72it/s]






4019it [04:24, 10.77it/s]






4029it [04:26,  9.68it/s]






4037it [04:27,  8.69it/s]






4049it [04:28,  8.63it/s]






4066it [04:30,  9.20it/s]






4080it [04:31,  9.71it/s]






4100it [04:33, 10.27it/s]






4127it [04:34, 12.25it/s]






4132it [

9902it [11:23,  8.49it/s]






9909it [11:24,  7.33it/s]






9935it [11:26,  8.99it/s]






9947it [11:27,  9.03it/s]






9994it [11:28, 11.65it/s]






10064it [11:29, 15.19it/s]






10085it [11:31, 15.12it/s]






10176it [11:32, 19.63it/s]






10233it [11:34, 23.64it/s]






10237it [11:35,  6.57it/s]






10266it [11:37,  8.03it/s]






10283it [11:38,  8.91it/s]






10289it [11:40,  6.09it/s]






10296it [11:41,  5.96it/s]






10304it [11:43,  6.06it/s]






10339it [11:44,  7.87it/s]






10369it [11:45,  9.82it/s]






10394it [11:50,  7.97it/s]






10401it [11:51,  6.27it/s]






10403it [11:53,  3.36it/s]






10404it [11:55,  1.06s/it]






10405it [11:57,  1.12s/it]






10407it [11:58,  1.00s/it]






10408it [11:59,  1.05s/it]






10417it [12:01,  1.29it/s]






10419it [12:02,  1.36it/s]






10421it [12:03,  1.42it/s]






10429it [12:04,  1.85it/s]






10434it [12:06,  2.09it/s]






10442it [12:07,  2.61it/s]






10452it [12:09,

14854it [18:47, 10.86it/s]






14857it [18:49,  4.51it/s]






14861it [18:50,  3.66it/s]






14883it [18:51,  4.77it/s]






14891it [18:53,  4.67it/s]






14899it [18:54,  5.08it/s]






14929it [18:56,  6.58it/s]






14949it [18:57,  7.96it/s]






14976it [18:59,  9.51it/s]






14994it [19:00, 10.68it/s]






15004it [19:01,  9.65it/s]






15040it [19:03, 11.90it/s]






15052it [19:04, 10.81it/s]






15065it [19:05, 10.23it/s]






15070it [19:07,  6.70it/s]






15085it [19:08,  7.76it/s]






15087it [19:09,  3.72it/s]






15088it [19:11,  1.22it/s]






15113it [19:12,  1.70it/s]






15157it [19:14,  2.38it/s]






15165it [19:15,  2.93it/s]






15171it [19:17,  3.13it/s]






15173it [19:18,  2.21it/s]






15179it [19:20,  2.56it/s]






15187it [19:21,  3.03it/s]






15205it [19:22,  3.95it/s]






15211it [19:24,  4.22it/s]






15220it [19:25,  4.82it/s]






15222it [19:26,  2.93it/s]






15224it [19:27,  2.41it/s]






15226it [1

19620it [27:12,  1.98it/s]






19625it [27:14,  2.29it/s]






19629it [27:15,  2.51it/s]






19635it [27:19,  2.04it/s]






19661it [27:20,  2.80it/s]






19675it [27:22,  3.57it/s]






19684it [27:23,  4.18it/s]






19709it [27:24,  5.49it/s]






19734it [27:25,  7.03it/s]






19755it [27:27,  8.09it/s]






19784it [27:30,  8.94it/s]






19801it [27:31,  9.95it/s]






19824it [27:33,  9.87it/s]






19836it [27:35,  9.26it/s]






19844it [27:36,  8.13it/s]






19851it [27:37,  7.15it/s]






19855it [27:39,  4.94it/s]






19891it [27:43,  5.64it/s]






19900it [27:46,  4.12it/s]






19936it [27:48,  5.34it/s]






19946it [27:50,  5.71it/s]






19953it [27:51,  5.39it/s]






19974it [27:53,  6.74it/s]






19997it [27:54,  8.16it/s]






20007it [27:56,  7.92it/s]






20011it [27:57,  5.03it/s]






20026it [27:58,  6.11it/s]






20064it [27:59,  8.02it/s]






20072it [28:02,  5.81it/s]






20078it [28:05,  3.81it/s]






20087it [2

23179it [35:28, 10.05it/s]






23206it [35:30, 11.23it/s]






23214it [35:31,  9.12it/s]






23255it [35:33, 11.54it/s]






23298it [35:34, 14.18it/s]






23301it [35:36,  4.73it/s]






23304it [35:37,  3.60it/s]






23306it [35:38,  2.33it/s]






23309it [35:40,  2.37it/s]






23315it [35:41,  2.75it/s]






23319it [35:42,  2.80it/s]






23340it [35:45,  3.49it/s]






23365it [35:46,  4.63it/s]






23378it [35:48,  5.50it/s]






23395it [35:49,  6.71it/s]






23411it [35:50,  7.73it/s]






23413it [35:51,  3.75it/s]






23422it [35:53,  4.36it/s]






23430it [35:54,  4.83it/s]






23438it [35:55,  5.18it/s]






23458it [35:57,  6.26it/s]






23469it [35:58,  6.69it/s]






23497it [35:59,  8.44it/s]






23500it [36:01,  4.93it/s]






23506it [36:02,  4.71it/s]






23519it [36:03,  5.63it/s]






23532it [36:05,  6.55it/s]






23543it [36:06,  7.14it/s]






23551it [36:07,  7.03it/s]






23556it [36:08,  5.62it/s]






23583it [3

26552it [42:37,  4.93it/s]






26556it [42:38,  4.15it/s]






26560it [42:39,  3.69it/s]






26582it [42:41,  4.80it/s]






26584it [42:42,  2.63it/s]






26590it [42:44,  2.92it/s]






26603it [42:45,  3.64it/s]






26608it [42:46,  3.78it/s]






26612it [42:48,  3.66it/s]






26614it [42:49,  2.55it/s]






26647it [42:51,  3.45it/s]






26648it [42:52,  1.71it/s]






26708it [42:53,  2.40it/s]






26710it [42:55,  2.08it/s]






26712it [42:56,  1.88it/s]






26743it [42:57,  2.59it/s]






26814it [42:59,  3.63it/s]






26837it [43:00,  4.79it/s]






26840it [43:01,  3.62it/s]






26856it [43:02,  4.61it/s]






26859it [43:04,  3.60it/s]






26887it [43:05,  4.80it/s]






26912it [43:06,  6.26it/s]






26914it [43:07,  3.09it/s]






26918it [43:09,  2.96it/s]






26924it [43:10,  3.30it/s]






26935it [43:12,  3.84it/s]






26937it [43:13,  2.54it/s]






26950it [43:15,  3.30it/s]






26951it [43:16,  1.58it/s]






26953it [4

29934it [49:49,  2.85it/s]






29964it [49:50,  3.87it/s]






29977it [49:51,  4.78it/s]






30025it [49:52,  6.49it/s]






30036it [49:54,  6.98it/s]






30060it [49:55,  8.52it/s]






30062it [49:56,  3.66it/s]






30093it [49:57,  4.92it/s]






30102it [49:59,  5.38it/s]






30107it [50:00,  4.85it/s]






30114it [50:01,  5.00it/s]






30120it [50:04,  3.66it/s]






30137it [50:05,  4.72it/s]






30182it [50:06,  6.39it/s]






30190it [50:09,  4.72it/s]






30203it [50:10,  5.60it/s]






30221it [50:12,  6.47it/s]






30245it [50:14,  8.01it/s]






30273it [50:15,  9.72it/s]






30287it [50:16, 10.21it/s]






30337it [50:17, 13.19it/s]






30346it [50:19, 10.67it/s]






30363it [50:20, 11.62it/s]






30372it [50:21, 10.09it/s]






30375it [50:22,  5.30it/s]






30408it [50:23,  6.98it/s]






30420it [50:25,  7.37it/s]






30436it [50:26,  8.41it/s]






30438it [50:28,  3.18it/s]






30464it [50:29,  4.28it/s]






30477it [5

33342it [57:07,  5.92it/s]






33344it [57:08,  3.20it/s]






33350it [57:10,  3.48it/s]






33365it [57:11,  4.41it/s]






33367it [57:13,  2.09it/s]






33369it [57:15,  1.77it/s]






33388it [57:16,  2.41it/s]






33395it [57:17,  2.87it/s]






33396it [57:18,  1.58it/s]






33399it [57:20,  1.58it/s]






33402it [57:22,  1.72it/s]






33403it [57:24,  1.02it/s]






33411it [57:25,  1.32it/s]






33412it [57:27,  1.03it/s]






33426it [57:28,  1.42it/s]






33428it [57:29,  1.47it/s]






33430it [57:31,  1.53it/s]






33439it [57:32,  2.01it/s]






33459it [57:33,  2.73it/s]






33462it [57:34,  2.68it/s]






33468it [57:35,  3.14it/s]






33476it [57:36,  3.75it/s]






33494it [57:38,  4.74it/s]






33501it [57:39,  4.74it/s]






33531it [57:41,  6.16it/s]






33551it [57:42,  7.42it/s]






33559it [57:44,  6.90it/s]






33561it [57:45,  3.14it/s]






33563it [57:46,  2.44it/s]






33585it [57:48,  3.30it/s]






33594it [5

KeyboardInterrupt: 

In [71]:
print(len([i for i in lat_res if i]))
print(len(failed_location_dict))

with open('lat_res.json', 'w') as outfile:
    json.dump(lat_res, outfile)
with open('lon_res.json', 'w') as outfile:
    json.dump(lon_res, outfile)
    

20092
123


In [72]:
df_musician.to_csv(r'musician_birth_final.csv', index=[0])

In [113]:
# Start here 11.30 18:32
import json

In [114]:
df_musician = pd.read_csv('musician_birth_final.csv', index_col=0)
df_musician.head()

df_musician = df_musician.drop(columns=['birth_place'])

print(len(df_musician[df_musician['birthplace'].isnull()]))
with open('lat_res.json', 'r') as infile: 
    lat_res = json.load(infile)
with open('lon_res.json', 'r') as infile:
    lon_res = json.load(infile)
with open('fail_dict.json', 'r') as infile:
    failed_location_dict = json.load(infile)

32719


In [115]:
print(len(failed_location_dict))

123


In [116]:
print(len([i for i in lon_res if i]))

20092


In [117]:
for i, location in tqdm(enumerate(df_musician['birthplace'])):
    if type(location) is str and not lat_res[i]:
        time.sleep(1)
        try:
            lat, lon = fetch_lat_lon2(location)
        except:
            failed_location_dict[i] = location
            continue
        lat_res[i] = lat
        lon_res[i] = lon



0it [00:00, ?it/s]

671it [00:01, 520.02it/s]

1331it [00:02, 511.46it/s]

1373it [00:03, 103.59it/s]

1466it [00:05, 93.52it/s] 

1495it [00:06, 49.47it/s]

1835it [00:07, 65.34it/s]

2047it [00:08, 80.01it/s]

2135it [00:10, 75.45it/s]

2276it [00:11, 83.65it/s]

2490it [00:12, 98.64it/s]

2519it [00:13, 51.85it/s]

2761it [00:15, 64.30it/s]

2787it [00:16, 36.72it/s]

2899it [00:18, 43.86it/s]

3115it [00:19, 56.64it/s]

3194it [00:20, 57.58it/s]

3446it [00:22, 72.97it/s]

3493it [00:23, 57.50it/s]

3508it [00:24, 27.48it/s]

4193it [00:25, 38.41it/s]

4337it [00:27, 48.24it/s]

5043it [00:28, 66.58it/s]

5369it [00:29, 85.61it/s]

5706it [00:30, 108.16it/s]

5976it [00:31, 127.34it/s]

6014it [00:33, 64.03it/s] 

6629it [00:34, 86.80it/s]

6770it [00:35, 92.10it/s]

7624it [00:37, 123.75it/s]

7731it [00:38, 105.14it/s]

8143it [00:39, 132.13it/s]

8187it [00:40, 73.84it/s] 

8217it [00:42, 47.18it/s]

8281it [00:43, 49.11it/s]

8347it [00:44, 51.10it/s]

8634it [00:45, 66.92it/

39787it [13:24,  8.17it/s]

39791it [13:25,  5.61it/s]

39832it [13:27,  7.44it/s]

39864it [13:28,  9.46it/s]

39891it [13:30, 10.72it/s]

39899it [13:31,  9.16it/s]

39919it [13:32, 10.35it/s]

39929it [13:34,  8.90it/s]

39931it [13:35,  3.87it/s]

39937it [13:36,  4.13it/s]

39941it [13:37,  3.87it/s]

39979it [13:39,  5.24it/s]

39989it [13:40,  5.80it/s]

40003it [13:41,  6.72it/s]

40020it [13:43,  7.85it/s]

40029it [13:44,  7.49it/s]

40032it [13:45,  4.53it/s]

40051it [13:46,  5.73it/s]

40053it [13:48,  3.37it/s]

40058it [13:49,  3.35it/s]

40073it [13:50,  4.27it/s]

40085it [13:52,  5.16it/s]

40100it [13:53,  6.29it/s]

40102it [13:54,  3.45it/s]

40105it [13:55,  3.04it/s]

40106it [13:56,  1.70it/s]

40115it [13:58,  2.18it/s]

40130it [13:59,  2.86it/s]

40144it [14:01,  3.66it/s]

40167it [14:02,  4.78it/s]

40174it [14:04,  4.34it/s]

40175it [14:07,  1.03it/s]

40176it [14:08,  1.12s/it]

40199it [14:09,  1.25it/s]

40201it [14:10,  1.34it/s]

40202it [14:12,  1.1

In [181]:
print(len(failed_location_dict))

270


In [123]:
cnt = 0
for index, i in enumerate(lon_res):
    if i is not None:
        val = df_musician.loc[index, "birthplace"]
        if type(val) == float and math.isnan(val):
            print(index)
print(df_musician.loc[53976, "birthplace"])

nan


In [169]:
cnt = 0
for key, value in failed_location_dict.items():
    if value == '':
        continue
    if not lat_res[int(key)]:
        cnt += 1
print(cnt)
print(len([i for i in lon_res if i]) + 147)

cnt = 0
for i in df_musician['birthplace']:
    if type(i) == float and math.isnan(i):
        continue
    cnt += 1
print(cnt)
    
    
print(len(df_musician['birthplace']) - len(df_musician[df_musician['birthplace'].isnull()]))
print(len(df_musician['birthplace']))
print(len(df_musician[df_musician['birthplace'].isnull()]))

259
21262
21262
21262
53981
32719


In [190]:
a = set(df_musician['birthplace'].index)
b = set(df_musician[df_musician['birthplace'].isnull()].index)
c = set([index for index, i in enumerate(lon_res) if i])
d = set([int(key) for key, value in failed_location_dict.items()])
cnt = 0
for name in d:
    if name in a:
        if name not in b and name not in c:
            cnt += 1 
    else:
        print(name)
print(cnt)

147


In [74]:
print(len(failed_location_dict))

270


In [191]:
df_musician['long'] = lon_res
df_musician['lat'] = lat_res
df_musician.to_csv(r'musician_birth_with_lat_long.csv', index=[0])

In [192]:
df_tracks = pd.read_csv('audio_features.csv', index_col=0)
df_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,track_id,popularity,year
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.707,0.00381,0.302,0.315,174.431,182400.0,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.764,0.00000,0.153,0.644,154.759,175987.0,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.733,0.00000,0.281,0.836,76.816,131733.0,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.878,0.00000,0.129,0.407,96.217,165560.0,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.617,0.00000,0.155,0.609,70.843,165280.0,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960


In [193]:
df_musician

,name,birthplace,long,lat
0,Etta James,Los Angeles,-118.244476,34.054935
1,Ella Fitzgerald,Newport News,-76.4321089,36.9786449
2,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.5709323,34.2001086
3,Neil Sedaka,Brooklyn,-73.9495823,40.6501038
4,Dean Martin,Steubenville,-80.6339638,40.3697905
...,...,...,...,...
53976,L.Dre,NaN,None,None
53977,Craig Stickland,NaN,None,None
53978,ギヴン,NaN,None,None
53979,Porch Upright,NaN,None,None


In [210]:
df = pd.merge(df_tracks, df_musician, left_on='artist_name', right_on='name', how='left')


        danceability  energy  key  loudness  mode  speechiness  acousticness  \
0              0.171   0.330  5.0    -9.699   1.0       0.0329       0.70700   
1              0.508   0.287  1.0   -12.472   1.0       0.0523       0.76400   
2              0.579   0.502  8.0    -7.570   1.0       0.0513       0.73300   
3              0.553   0.291  4.0   -10.426   0.0       0.0301       0.87800   
4              0.482   0.569  0.0    -5.897   1.0       0.0328       0.61700   
...              ...     ...  ...       ...   ...          ...           ...   
599995         0.352   0.895  5.0    -7.830   0.0       0.0626       0.00119   
599996         0.405   0.374  3.0    -9.227   0.0       0.0502       0.22200   
599997         0.420   0.874  6.0    -7.559   1.0       0.0540       0.00706   
599998         0.754   0.436  6.0    -9.183   0.0       0.3920       0.44900   
599999         0.757   0.586  1.0    -7.201   1.0       0.0489       0.09430   

        instrumentalness  liveness  val

In [243]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,time_signature,artist_name,track_name,track_id,popularity,year,name,birthplace,long,lat
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.70700,0.003810,0.302,0.3150,...,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960,Etta James,Los Angeles,-118.244476,34.054935
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.76400,0.000000,0.153,0.6440,...,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960,Ella Fitzgerald,Newport News,-76.4321089,36.9786449
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.73300,0.000000,0.281,0.8360,...,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960,Ella Fitzgerald,Newport News,-76.4321089,36.9786449
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.87800,0.000000,0.129,0.4070,...,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.5709323,34.2001086
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.61700,0.000000,0.155,0.6090,...,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.5709323,34.2001086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.352,0.895,5.0,-7.830,0.0,0.0626,0.00119,0.000008,0.974,0.4440,...,4.0,Foreigner,Head Games - Live,6fxUyaVit6FStloTOzorIr,21,2019,Foreigner,"New York City, New York, U.S.",-73.9679989451639,40.7490646
599996,0.405,0.374,3.0,-9.227,0.0,0.0502,0.22200,0.000000,0.107,0.0987,...,4.0,✦ pink cig ✦,Angeldust,2mj8hvQAp5RCi0K6SXe7YZ,58,2019,✦ pink cig ✦,None,None,None
599997,0.420,0.874,6.0,-7.559,1.0,0.0540,0.00706,0.000017,0.743,0.4760,...,4.0,Foreigner,Long Long Way from Home - Live,36Jl814jfojlZrX3v3Tvc8,21,2019,Foreigner,"New York City, New York, U.S.",-73.9679989451639,40.7490646
599998,0.754,0.436,6.0,-9.183,0.0,0.3920,0.44900,0.000000,0.104,0.4020,...,4.0,Vedo,Girls Need Love (V-Mix),79sYN8CwQMVwA4wNXg1RVt,55,2019,Vedo,None,None,None


In [219]:
df['birthplace'][df['birthplace'].isnull()] = df[df['birthplace'].isnull()]['birthplace'].apply(lambda x : None)

C:\Users\liyax\Miniconda3\envs\688midterm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [245]:
600000 - len(df[df['birthplace'].isnull() == df['lat'].isnull()])

2519

In [9]:
# Start here 11.30 23:39
df = pd.read_csv('feature_with_long_and_lat.csv', index_col=0)

In [10]:
df 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,time_signature,artist_name,track_name,track_id,popularity,year,name,birthplace,long,lat
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.70700,0.003810,0.302,0.3150,...,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960,Etta James,Los Angeles,-118.244476,34.054935
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.76400,0.000000,0.153,0.6440,...,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960,Ella Fitzgerald,Newport News,-76.432109,36.978645
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.73300,0.000000,0.281,0.8360,...,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960,Ella Fitzgerald,Newport News,-76.432109,36.978645
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.87800,0.000000,0.129,0.4070,...,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.570932,34.200109
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.61700,0.000000,0.155,0.6090,...,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.570932,34.200109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.352,0.895,5.0,-7.830,0.0,0.0626,0.00119,0.000008,0.974,0.4440,...,4.0,Foreigner,Head Games - Live,6fxUyaVit6FStloTOzorIr,21,2019,Foreigner,"New York City, New York, U.S.",-73.967999,40.749065
599996,0.405,0.374,3.0,-9.227,0.0,0.0502,0.22200,0.000000,0.107,0.0987,...,4.0,✦ pink cig ✦,Angeldust,2mj8hvQAp5RCi0K6SXe7YZ,58,2019,✦ pink cig ✦,NaN,NaN,NaN
599997,0.420,0.874,6.0,-7.559,1.0,0.0540,0.00706,0.000017,0.743,0.4760,...,4.0,Foreigner,Long Long Way from Home - Live,36Jl814jfojlZrX3v3Tvc8,21,2019,Foreigner,"New York City, New York, U.S.",-73.967999,40.749065
599998,0.754,0.436,6.0,-9.183,0.0,0.3920,0.44900,0.000000,0.104,0.4020,...,4.0,Vedo,Girls Need Love (V-Mix),79sYN8CwQMVwA4wNXg1RVt,55,2019,Vedo,NaN,NaN,NaN


In [11]:
df = df.dropna()

In [12]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,time_signature,artist_name,track_name,track_id,popularity,year,name,birthplace,long,lat
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.70700,0.003810,0.3020,0.315,...,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960,Etta James,Los Angeles,-118.244476,34.054935
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.76400,0.000000,0.1530,0.644,...,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960,Ella Fitzgerald,Newport News,-76.432109,36.978645
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.73300,0.000000,0.2810,0.836,...,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960,Ella Fitzgerald,Newport News,-76.432109,36.978645
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.87800,0.000000,0.1290,0.407,...,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.570932,34.200109
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.61700,0.000000,0.1550,0.609,...,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960,Sam Cooke,"Clarksdale, Mississippi, U.S.",-90.570932,34.200109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599991,0.611,0.649,10.0,-6.095,0.0,0.1330,0.01150,0.000000,0.0641,0.720,...,4.0,Sean Paul,When It Comes To You,55nUyfW8eCQsHrZ3jtkXLU,66,2019,Sean Paul,"Kingston, Jamaica",-76.792813,17.971215
599992,0.635,0.493,0.0,-8.631,1.0,0.0741,0.40100,0.000000,0.1420,0.207,...,4.0,Lolo Zouaï,High Highs to Low Lows,2aGNR8A6GnHqoM78T31CCC,57,2019,Lolo Zouaï,"Paris, France",2.351499,48.856610
599993,0.501,0.675,1.0,-5.080,1.0,0.0965,0.24100,0.000000,0.1980,0.501,...,4.0,Kelly Rowland,Love You More At Christmas Time,76jef0QPcSa0tY1Vu8ekhv,38,2019,Kelly Rowland,Atlanta,-84.390185,33.749099
599995,0.352,0.895,5.0,-7.830,0.0,0.0626,0.00119,0.000008,0.9740,0.444,...,4.0,Foreigner,Head Games - Live,6fxUyaVit6FStloTOzorIr,21,2019,Foreigner,"New York City, New York, U.S.",-73.967999,40.749065


In [13]:
import folium
from folium import plugins
from ipywidgets import interact
from folium.plugins import HeatMap

radius = widgets.IntSlider(min = 1960, max = 2019, step = 1, value = 1980, description = 'Year: ')

def response(year):    
    m = folium.Map(location=[48, -102], zoom_start=3)
    matrix = df[df['year'] == year][['lat', 'long']]
    m.add_child(plugins.HeatMap(matrix, radius = 15))
    return m

interact(response, year = radius)

interactive(children=(IntSlider(value=1980, description='Year: ', max=2019, min=1960), Output()), _dom_classes…

<function __main__.response(year)>

In [37]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,time_signature,artist_name,track_name,track_id,popularity,year,name,birthplace,long,lat
360000,0.682,0.7650,1.0,-5.021,0.0,0.0395,0.02680,0.000034,0.188,0.567,...,4.0,Sublime,Santeria,2hnMS47jN0etwvFPzYk11f,74,1996,Sublime,"Long Beach, California, U.S.",-118.191605,33.769016
360002,0.868,0.6460,1.0,-4.674,0.0,0.2880,0.30300,0.000000,0.284,0.670,...,4.0,Blackstreet,No Diggity,6MdqqkQ8sSC0WB4i8PyRuQ,78,1996,Blackstreet,"New York City, New York (state)|New York, Unit...",-73.986614,40.730646
360003,0.804,0.6960,7.0,-7.470,1.0,0.0899,0.00877,0.000985,0.314,0.657,...,4.0,Sublime,What I Got,3B4q6KbHbGV51HO3GznBFF,72,1996,Sublime,"Long Beach, California, U.S.",-118.191605,33.769016
360004,0.768,0.8640,11.0,-6.145,1.0,0.0324,0.10100,0.000000,0.162,0.891,...,4.0,Spice Girls,Wannabe,1Je1IMUlBXcx1Fz0WE7oPT,80,1996,Spice Girls,"London, England",-0.127647,51.507322
360007,0.715,0.8330,2.0,-5.379,1.0,0.1080,0.00631,0.000000,0.164,0.612,...,4.0,Mark Morrison,Return of the Mack,3jDdpx9PMlfMBS5tOBHFm9,73,1996,Mark Morrison,"Leicester, England",-1.133079,52.636140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369991,0.727,0.6750,1.0,-6.273,1.0,0.0492,0.01630,0.000004,0.098,0.806,...,4.0,Spice Girls,Say You'll Be There,1qXsLBY6TfX8xPVUBw36E7,25,1996,Spice Girls,"London, England",-0.127647,51.507322
369994,0.467,0.4750,0.0,-16.575,0.0,0.0455,0.97600,0.017500,0.151,0.850,...,4.0,Fats Waller,I'm Crazy 'Bout My Baby (And My Baby's Crazy '...,1vgLNJD8NcAXqAmzAzJUrf,14,1996,Fats Waller,New York City,-73.986614,40.730646
369995,0.648,0.0841,0.0,-22.554,1.0,0.8080,0.67600,0.000000,0.120,0.657,...,5.0,Rakim,Outro,1xiqGaMeZqLFY9TpA07k4F,20,1996,Rakim,New York City,-73.986614,40.730646
369996,0.855,0.1850,6.0,-13.255,1.0,0.0618,0.89100,0.000191,0.378,0.652,...,4.0,Wilco,Losing Interest,1yMEf1YsG2Mdr2Ffmku7zD,16,1996,Wilco,"Chicago, Illinois, United States|U.S.",-79.876601,40.273544
